In [30]:
%matplotlib inline

from __future__ import print_function
import math
import itertools
import numpy
import numpy.linalg
import matplotlib.pyplot as plt

In [31]:
U = 1.0
J = 0.01
mu = 0.5
print('J/U:  %f' % (J / U))
print('mu/U: %f' % (mu / U))

J/U:  0.010000
mu/U: 0.500000


In [32]:
L = 5
nmax = 3
nstates = (nmax + 1) ** L
states = numpy.empty((nstates, L), dtype=int)
print('#sites:                   %i' % L)
print('nmax:                     %i' % nmax)
print('Hilbert-space dimensions: %i' % nstates)

#sites:                   5
nmax:                     3
Hilbert-space dimensions: 1024


In [33]:
istate=0
for state in itertools.product(range(nmax + 1), repeat=L):
    states[istate, :] = state[:]
    istate += 1
print('States enumeration completed')

States enumeration completed


In [45]:
# Fill matrix H
def build_H(states, U, J, mu):
    H = numpy.zeros((nstates,nstates))
    for istate1, state1 in enumerate(states):
        # Diagonal terms
        for site in xrange(L):
            H[istate1, istate1] += 0.5 * U * state1[site] * (state1[site] - 1.0)
            H[istate1, istate1] -= mu * state1[site]

        # Off-diagonal terms
        for istate2, state2 in enumerate(states):
            H[istate1, istate2] = 0.0
            for site in xrange(L):

                state1_site = state1[site]
                state2_site = state2[site]
                state1_site_p1 = state1[(site + 1) % L]
                state2_site_p1 = state2[(site + 1) % L]

                if ((state1_site == (state2_site + 1)) and
                    (state1_site_p1 == (state2_site_p1 - 1))):
                    H[istate1, istate2] -= J * math.sqrt(state1_site * (state1_site_p1 + 1.0))
                    assert istate1 != istate2
                if ((state1_site_p1 == (state2_site_p1 + 1)) and
                    (state1_site == (state2_site - 1))):
                    H[istate1, istate2] -= J * math.sqrt(state1_site_p1 * (state1_site + 1.0))
    print('All entries of H filled.')
    print('Is the matrix H symmetric?', numpy.allclose(H, H.T))
    return H
H = build_H(states, U, J, mu)

All entries of H filled.
Is the matrix H symmetric? True


In [46]:
if nstates < 100:
    plt.matshow(H, origin='lower')
    plt.title('Hamiltonian matrix')
    plt.gca().xaxis.tick_bottom()
    plt.colorbar()
    plt.xlabel('States')
    plt.ylabel('States')

In [47]:
evals, evecs = numpy.linalg.eigh(H)
indices = evals.argsort()
evals = evals[indices]
evecs = (evecs.T)[indices]
for ind in xrange(5):
    print('%i-th eigenvalue: %+f' % (ind, evals[ind]))

0-th eigenvalue: -7.779445
1-th eigenvalue: -2.424119
2-th eigenvalue: -2.209857
3-th eigenvalue: -2.209857
4-th eigenvalue: -2.139946


In [48]:
def average_on_site_density(site, coefficients, states):
    assert abs((coefficients ** 2).sum() - 1.0) < 1e-10
    dens = 0.0
    for istate, state in enumerate(states):
        dens += (coefficients[istate] ** 2) * state[site]
    return dens

for istate in xrange(min(nstates, 25)):
    ns = [average_on_site_density(site, evecs[istate], states) for site in xrange(min(L, 3))]
    print('[%2i | E=%+f] <n_i>:' % (istate, evals[istate]), ns)


[ 0 | E=-7.779445] <n_i>: [1.6595157621122416, 1.6595157621122436, 1.6595157621122449]
[ 1 | E=-2.424119] <n_i>: [1.6943024682972816, 1.6943024682972829, 1.6943024682972816]
[ 2 | E=-2.209857] <n_i>: [1.5920600145171357, 1.6512692139288161, 1.6512692222712579]
[ 3 | E=-2.209857] <n_i>: [1.6777483774959765, 1.618539178084295, 1.6185391697418567]
[ 4 | E=-2.139946] <n_i>: [1.7347726313786185, 1.7347726313786092, 1.7347726313786083]
[ 5 | E=-2.083776] <n_i>: [1.8145987181193461, 1.860278013474473, 1.8602780134744734]
[ 6 | E=-2.083776] <n_i>: [1.880706415390158, 1.835027120035017, 1.8350271200350192]
[ 7 | E=-2.064665] <n_i>: [1.6218260095781702, 1.7892594716539687, 1.7892594716539678]
[ 8 | E=-2.064665] <n_i>: [1.7143809510720462, 1.5469474889962667, 1.5469474889962618]
[ 9 | E=-1.770475] <n_i>: [1.5702435772141368, 1.7237948782041499, 1.7237948781854977]
[10 | E=-1.770475] <n_i>: [1.6551246487677154, 1.5015733477777016, 1.5015733477963524]
[11 | E=-1.635770] <n_i>: [1.3461199831735984, 